## Models

In [10]:
import model_CIFAR10
import model_CIFAR100
import torch
from pytorchcv.model_provider import get_model as ptcv_get_model

model_1 = model_CIFAR10.inceptionv4()
model_1.load_state_dict(torch.load('./models_param/inceptionv4_cifar10.pth'))
model_2 = model_CIFAR10.resnet152()
model_2.load_state_dict(torch.load('./models_param/resnet152_cifar10.pth'))
model_3 = model_CIFAR10.vgg19_bn()
model_3.load_state_dict(torch.load('./models_param/vgg19_cifar10.pth'))

model_4 = model_CIFAR100.inceptionv4()
model_4.load_state_dict(torch.load('./models_param/inceptionv4_cifar100.pth'))
model_5 = model_CIFAR100.resnet152()
model_5.load_state_dict(torch.load('./models_param/resnet152_cifar100.pth'))
model_6 = model_CIFAR100.vgg19_bn()
model_6.load_state_dict(torch.load('./models_param/vgg19_cifar100.pth'))

cifar10_models = [model_1.cuda(), model_2.cuda(), model_3.cuda()]
cifar100_models = [model_4.cuda(), model_5.cuda(), model_6.cuda()]

## Dataset

### CIFAR 10

In [11]:
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler

X_size = 300

CIFAR10_mean = (0.491, 0.482, 0.447)
CIFAR10_std = (0.202, 0.199, 0.201)

transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=CIFAR10_mean, std=CIFAR10_std),
])

dataset_cifar10 = torchvision.datasets.CIFAR10(
    root='./cifar-10', train=True, download=True, transform=transform_cifar10)
dataset_cifar10_test = torchvision.datasets.CIFAR10(
    root='./cifar-10', train=False, download=True, transform=transform_cifar10)

indicies = list(range(len(dataset_cifar10)))
np.random.seed(12)
np.random.shuffle(indicies)

dataloader_cifar10 = torch.utils.data.DataLoader(
    dataset_cifar10, batch_size=16, num_workers=2, sampler=SubsetRandomSampler(indicies[:X_size]))
dataloader_cifar10_test = torch.utils.data.DataLoader(
    dataset_cifar10_test, batch_size=16, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


### CIFAR 100

In [12]:
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler

X_size = 1000

cifar100_mean = (0.507, 0.487, 0.441)
cifar100_std = (0.267, 0.256, 0.276)

transform_cifar100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar100_mean, std=cifar100_std),
])

dataset_cifar100 = torchvision.datasets.CIFAR100(
    root='./cifar-100', train=True, download=True, transform=transform_cifar100)
dataset_cifar100_test = torchvision.datasets.CIFAR100(
    root='./cifar-100', train=False, download=True, transform=transform_cifar100)

indicies = list(range(len(dataset_cifar100)))
np.random.seed(12)
np.random.shuffle(indicies)

dataloader_cifar100 = torch.utils.data.DataLoader(
    dataset_cifar100, batch_size=16, num_workers=2, sampler=SubsetRandomSampler(indicies[:X_size]))
dataloader_cifar100_test = torch.utils.data.DataLoader(
    dataset_cifar100_test, batch_size=16, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [13]:
for model_idx in range(3):
    cifar100_models[model_idx].cuda()
    cifar100_models[model_idx].eval()
    total = 0
    correct = 0
    for (inputs, targets) in dataloader_cifar100_test:
        outputs = cifar100_models[model_idx](inputs.cuda())
        _, predict = outputs.max(1)
        total += targets.size(0)
        correct += predict.cpu().eq(targets).sum().item()
    print(correct/total)

0.782
0.7954
0.7165


## UAP

In [14]:
from deepfool import deepfool
from tqdm import tqdm

delta = 0.4
eps = 16
eps = eps/255/torch.Tensor([0.2023, 0.1994, 0.2010])

uap = torch.zeros(dataset_cifar10[0][0].shape).cuda()

for model_idx in range(3):
    cifar10_models[model_idx].cuda()



for j, model in enumerate(cifar10_models):
    model.eval()
    min_acc = 100    
    acc = 100
    iters=0
    while acc > delta:
        progress = tqdm(total=X_size, position=0, leave=True, bar_format='{desc:<5.5}{percentage:3.0f}%|{bar:60}{r_bar}')
        for inputs, labels in dataloader_cifar10:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs+uap.cuda())
            _, predict = outputs.max(1)
            for i in range(inputs.shape[0]):
                if predict[i] == labels[i]:
                    uap = uap.cuda()
                    r,iter_df,_,_,_ = deepfool(inputs[i]+uap, model, outputs.shape[-1], overshoot=0.02, max_iter=50)
                    uap = uap.cpu()
                    if iter_df < 50-1:
                        uap[0,:,:] = torch.clamp(uap[0,:,:] + r[0, 0,:,:], min=-eps[0].item(), max=eps[0].item())
                        uap[1,:,:] = torch.clamp(uap[1,:,:] + r[0, 1,:,:], min=-eps[1].item(), max=eps[1].item())
                        uap[2,:,:] = torch.clamp(uap[2,:,:] + r[0, 2,:,:], min=-eps[2].item(), max=eps[2].item())
                progress.update(1)

        total = 0
        correct = 0

        for (inputs, targets) in dataloader_cifar10_test:
            outputs = model(inputs.cuda()+uap.cuda())
            _, predict = outputs.max(1)
            total += targets.size(0)
            correct += predict.cpu().eq(targets).sum().item()

        acc = correct/total
        print('Accuracy after UAP: %.3f %%' % (acc*100))
        progress.close()
        iters+=1
        if iters > 3:
            break
        if acc < min_acc:
            min_acc = acc
            torch.save(uap.cpu().detach(), 'model_'+str(j+1)+'_uap_cifar10.pth')
    print("============= Model %d UAP generated ==============" % (j))

       0%|                                                            | 0/1000 [00:00<?, ?it/s]

Accuracy after UAP: 35.060 %
============= Model 0 UAP generated ==============


       0%|                                                            | 0/1000 [00:00<?, ?it/s]

Accuracy after UAP: 37.370 %
============= Model 1 UAP generated ==============


       0%|                                                            | 0/1000 [00:00<?, ?it/s]

...

Accuracy after UAP: 45.070 %


       0%|                                                            | 0/1000 [00:00<?, ?it/s]

Accuracy after UAP: 42.770 %


      30%|██████████████████                                          | 300/1000 [00:37<01:26,  8.10it/s]

Accuracy after UAP: 34.910 %
============= Model 2 UAP generated ==============


In [8]:
from deepfool import deepfool
from tqdm import tqdm

delta = 0.3
eps = 10
eps = eps/255/torch.Tensor(cifar100_std)

uap = torch.zeros(dataset_cifar100[0][0].shape).cuda()
acc = 100

for model_idx in range(3):
    cifar100_models[model_idx].cuda()



for j, model in enumerate(cifar100_models):
    if j == 0:
        continue
    model.eval()
    iters=0
    min_acc = 100    
    while acc > delta:
        progress = tqdm(total=X_size, position=0, leave=True, bar_format='{desc:<5.5}{percentage:3.0f}%|{bar:60}{r_bar}')
        for inputs, labels in dataloader_cifar100:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs+uap.cuda())
            _, predict = outputs.max(1)
            for i in range(inputs.shape[0]):
                if predict[i] == labels[i]:
                    uap = uap.cuda()
                    r,iter_df,_,_,_ = deepfool(inputs[i]+uap, model, outputs.shape[-1], overshoot=0.02, max_iter=50)
                    uap = uap.cpu()
                    if iter_df < 50-1:
                        uap[0,:,:] = torch.clamp(uap[0,:,:] + r[0, 0,:,:], min=-eps[0].item(), max=eps[0].item())
                        uap[1,:,:] = torch.clamp(uap[1,:,:] + r[0, 1,:,:], min=-eps[1].item(), max=eps[1].item())
                        uap[2,:,:] = torch.clamp(uap[2,:,:] + r[0, 2,:,:], min=-eps[2].item(), max=eps[2].item())
                progress.update(1)

        total = 0
        correct = 0

        for (inputs, targets) in dataloader_cifar100_test:
            outputs = model(inputs.cuda()+uap.cuda())
            _, predict = outputs.max(1)
            total += targets.size(0)
            correct += predict.cpu().eq(targets).sum().item()

        acc = correct/total
        print('Accuracy after UAP: %.3f %%' % (acc*100))
        progress.close()
        iters+=1
        if iters > 2:
            break
        if acc < min_acc:
            min_acc = acc
            torch.save(uap.cpu().detach(), 'model_'+str(j+1)+'_uap_cifar100.pth')
    print("============= Model %d UAP generated ==============" % (j))

       0%|                                                            | 0/1000 [00:00<?, ?it/s]

Accuracy after UAP: 43.060 %


       0%|                                                            | 0/1000 [00:00<?, ?it/s]

Accuracy after UAP: 41.350 %


       0%|                                                            | 0/1000 [00:00<?, ?it/s]

...

Accuracy after UAP: 46.360 %


       0%|                                                            | 0/1000 [00:00<?, ?it/s]

Accuracy after UAP: 45.010 %


     100%|████████████████████████████████████████████████████████████| 1000/1000 [19:32<00:00,  1.17s/it]

Accuracy after UAP: 48.780 %
============= Model 2 UAP generated ==============


In [7]:
uap = torch.load("./model_1_uap_cifar100.pth")
for model in cifar100_models:#cifar10_models:
    model.eval()
    total = 0
    correct = 0

    
    
    for (inputs, targets) in dataloader_cifar100_test:
        outputs = model(inputs.cuda()+uap.cuda())
        _, predict = outputs.max(1)
        total += targets.size(0)
        correct += predict.cpu().eq(targets).sum().item()
    acc = correct/total
    print('Accuracy after UAP: %.3f %%' % (acc*100))

Accuracy after UAP: 36.440 %
Accuracy after UAP: 43.370 %
Accuracy after UAP: 37.840 %
